In [1]:
import pandas as pd

Сделаем один датасет со всеми необходимыми для анализа полями

In [2]:
artem_data = 'data/artem_data.xlsx'
students_data = 'data/students_data.xlsx'
diag_work_data = 'data/diagwork_data.csv'
salary_data = 'data/salaries.xlsx'

In [3]:
artem_data_df = pd.read_excel(artem_data)
students_data_df = pd.read_excel(students_data)
diag_work_data_df = pd.read_csv(diag_work_data, low_memory=False)
salary_data_df = pd.read_excel(salary_data)

D:\diploma\venv\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Добавим регион

In [4]:
df = pd.merge(
    artem_data_df, 
    students_data_df[['№', 'Регион_т']], 
    left_on='Number',
    right_on='№', 
    how='left'
)
del df['№']

Добавим id школы

In [5]:
df = pd.merge(
    df, 
    diag_work_data_df[['№', 'Логин_ОО']], 
    left_on='Number', 
    right_on='№', 
    how='left'
)
del df['№']

Добавим зарплату в регионе

In [6]:
df = pd.merge(
    df, salary_data_df, on='Регион_т', how='left'
)

Добавим первичный балл диагностической работы

In [7]:
df = pd.merge(
    df, 
    diag_work_data_df[['№', 'ПБ']], 
    left_on='Number', 
    right_on='№', 
    how='left'
)
del df['№']

Переименуем столбцы

In [8]:
df.rename(
    columns={
        'Region': 'District', 
        'Регион_т': 'Region', 
        'Зарплата': 'Salary'
    }, 
    inplace=True
)

Изменим порядок полей

In [9]:
new_order = [
    'Number', 'Логин_ОО', 'Class', 'Sex', 'Rus', 'Math', 'ПБ',
    'Family', 'District', 'Region', 'Salary',
    'History Knowledge Index', 'Family Life Index', 'Culture Index',
]
df = df[new_order]

Объединим "Северо-Кавказский федеральный округ" и "Южный федеральный округ"

In [10]:
df['District'] = df['District'].replace(
    'Северо-Кавказский федеральный округ', 'Южный федеральный округ'
)

Уберем из выборки данные, где нет данных о семье, об оценке

In [11]:
df = df[df['Family'] != 'Нет четкого ответа']
df = df[df['Math'] != 'нет отметки']
df = df[df['Rus'] != 'нет отметки']

Объединим расширенные и полные школы, переименуем "Многодетная" на "Многодетная полная"

In [12]:
df['Family'] = df['Family'].replace('Расширенная', 'Полная')
df['Family'] = df['Family'].replace('Многодетная', 'Многодетная полная')

Отсортируем данные по логину для удобства просмотра

In [13]:
df = df.sort_values(by='Логин_ОО', ascending=True)

Посмотрим данные

In [14]:
df  

,Number,Логин_ОО,Class,Sex,Rus,Math,ПБ,Family,District,Region,Salary,History Knowledge Index,Family Life Index,Culture Index
651,1904174,edu030331,6,Мальчики,4,5,10,Полная,Дальневосточный федеральный округ,Республика Бурятия,37297.2,2,2,2
211,2073399,edu030331,8,Девочки,4,4,12,Полная,Дальневосточный федеральный округ,Республика Бурятия,37297.2,2,3,2
223,2073406,edu030331,8,Девочки,5,4,6,Неполная,Дальневосточный федеральный округ,Республика Бурятия,37297.2,2,1,2
227,2073398,edu030331,8,Девочки,4,3,12,Полная,Дальневосточный федеральный округ,Республика Бурятия,37297.2,2,2,2
93,2120327,edu030331,8,Девочки,4,4,10,Многодетная полная,Дальневосточный федеральный округ,Республика Бурятия,37297.2,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39480,2081261,edu893072,8,Девочки,4,4,7,Многодетная полная,Уральский федеральный округ,Ямало-Ненецкий автономный округ,100527.6,2,1,2
39479,2081265,edu893072,8,Мальчики,3,3,11,Неполная,Уральский федеральный округ,Ямало-Ненецкий автономный округ,100527.6,1,2,2
39478,2081246,edu893072,8,Мальчики,3,3,8,Полная,Уральский федеральный округ,Ямало-Ненецкий автономный округ,100527.6,2,2,2
39489,1921602,edu893072,6,Мальчики,4,4,8,Многодетная полная,Уральский федеральный округ,Ямало-Ненецкий автономный округ,100527.6,2,2,2


Сохраним датасет в файл "data_people.csv"

In [15]:
data = 'data/data_people.csv'
df.to_csv(data, index=False)